## File with generalized Model 1 that we will use to optimize all key routes

### Packages

In [221]:
using CSV, DataFrames, JuMP, Gurobi, Plots

### Functions

In [233]:
function model_route(route_id, solve_category, verbose=false)
    # function to optimize any route
    println("optimizing route " * string(route_id))

    #data = CSV.read("data_route1.csv", DataFrame)
    # read in data
    data = CSV.read("key_routes_grouped/data_route" * string(route_id) * ".csv", DataFrame)
    
    # get the columns we need from the data
    on_data = data[:, 3]
    off_data = data[:, 4]
    load_data = data[:, 5]
    x = data[:, 6]
    y = data[:, 7]
    n = length(x)

    # precalculate distance matrix from x and y
    dist = zeros(n, n)
    for i in 1:length(x)
        for j in 1:length(x)
            dist[i, j] = sqrt((x[i] - x[j])^2 + (y[i] - y[j])^2)
        end
    end
    dist = Matrix(dist)

    # initialize lambda values and arrays to store results
    lambda_values = 0:0.05:1
    co2_values = zeros(length(lambda_values))
    profit_values = zeros(length(lambda_values))
    nb_stops = zeros(length(lambda_values))
    co2_change = zeros(length(lambda_values))
    profit_change = zeros(length(lambda_values))

    # calcalate current co2 and profit
    current_co2 = sum(load_data[i] * dist[i, i+1] for i in 1:n-1) + n
    current_profit = sum(on_data[i] for i in 1:n)

    # for testing purposes
    println("on data: ", on_data)
    println("off data: ", off_data)
    println("load data: ", load_data)
    println("x: ", x)
    println("y: ", y)
    println("n: ", n)
    println("dist: ", dist)
    println("current_co2: ", current_co2)
    println("current_profit: ", current_profit)

    # only do this if required by type
    if solve_category == "both" || solve_category == "general"
        # iterate across all lambdas
        for i in 1:length(lambda_values)
            println("optimizing for lambda=", lambda_values[i])

            # run optimization and retrieve objective
            z, load, off, objective = bus_model(on_data, off_data, load_data, x, y, n, dist, lambda_values[i])
            z_opt = abs.(round.(value.(z)))
            load_opt = value.(load)
            off_opt = value.(off)

            # calculate model co2 and profit
            model_co2 = sum(load_opt[i] * dist[i, i+1] for i in 1:n-1) + sum(z_opt[i] for i in 1:n)
            model_profit = sum(z_opt[i] * on_data[i] for i in 1:n)

            # add data to arrays
            co2_values[i] = model_co2
            profit_values[i] = model_profit
            nb_stops[i] = sum(z_opt)

            # calculate model co2 and profit improvements
            co2_change[i] = round((current_co2 - model_co2) / current_co2 * 100, digits=1)
            profit_change[i] = abs(round((model_profit - current_profit) / current_profit * 100, digits=1))

            # add z_opt to dataframe
            data[!, "lambda_" * string(lambda_values[i])] = z_opt

            if verbose
                model_stats(z, load, load_data, on_data, off, dist, n)
            end
        end

        # put all results into a nice dataframe
        results_df = DataFrame(lambda = lambda_values, n_stops = nb_stops, co2 = co2_values, profit = profit_values, co2_pct_reduction = co2_change, profit_pct_decrease = profit_change)

        # write csvs
        CSV.write("key_routes_grouped_results/route_" * string(route_id) * "_with_results.csv", data)
        CSV.write("key_routes_grouped_results/route_" * string(route_id) * "_opt_results.csv", results_df)
    end
    
    if solve_category == "both" || solve_category == "no_consecutive_drops"
        # iterate across all lambdas
        for i in 1:length(lambda_values)
            println("optimizing for lambda=", lambda_values[i])

            # run optimization and retrieve objective
            z, load, off, objective = bus_model_consecutive_constraint(on_data, off_data, load_data, x, y, n, dist, lambda_values[i])
            z_opt = abs.(round.(value.(z)))
            load_opt = value.(load)
            off_opt = value.(off)

            # calculate model co2 and profit
            model_co2 = sum(load_opt[i] * dist[i, i+1] for i in 1:n-1) + sum(z_opt[i] for i in 1:n)
            model_profit = sum(z_opt[i] * on_data[i] for i in 1:n)

            # add data to arrays
            co2_values[i] = model_co2
            profit_values[i] = model_profit
            nb_stops[i] = sum(z_opt)

            # calculate model co2 and profit improvements
            co2_change[i] = round((current_co2 - model_co2) / current_co2 * 100, digits=1)
            profit_change[i] = abs(round((model_profit - current_profit) / current_profit * 100, digits=1))

            # add z_opt to dataframe
            data[!, "lambda_" * string(lambda_values[i])] = z_opt

            if verbose
                model_stats(z, load, load_data, on_data, off, dist, n)
            end
        end

        # put all results into a nice dataframe
        results_df = DataFrame(lambda = lambda_values, n_stops = nb_stops, co2 = co2_values, profit = profit_values, co2_pct_reduction = co2_change, profit_pct_decrease = profit_change)

        # write csvs
        CSV.write("key_routes_grouped_results/route_" * string(route_id) * "_with_results_no_consecutive_drops.csv", data)
        CSV.write("key_routes_grouped_results/route_" * string(route_id) * "_opt_results_no_consecutive_drops.csv", results_df)
    end

    return results_df, data
end

function bus_model(on_data, off_data, load_data, x, y, n, dist, lambda)
    # function to optimize the bus route for a given lambda

    model = Model(Gurobi.Optimizer)
    set_attribute(model, "OutputFlag", 0)

    @variable(model, z[1:n], Bin)
    @variable(model, load[1:n] >= 0)
    @variable(model, off[1:n] >= 0)

    @constraint(model, [i in 1:n], load[i] <= 40)
    # load at bus stop j = load at bus stop j-1 + z_j * (number_go_on_at_stop_j - number_go_off_at_j-1)
    @constraint(model, load[1] == load_data[1])
    @constraint(model, load[n] <= 1)    # not sure here? equal to load_data or not?
    @constraint(model, [i in 2:n], load[i] == load[i-1] + z[i] * (on_data[i] - off[i-1]))
    # If a stop is removed, people go go off at the next available stop off_j = off_data_j + (1-z_{j-1})*off_j-1
    @constraint(model, off[1] == off_data[1])
    @constraint(model, [i in 2:n], off[i] == off_data[i] + (1-z[i-1]) * off[i-1])
    # bus has to stop at first and last stop
    @constraint(model, z[1] == 1)
    @constraint(model, z[n] == 1)

    # CO2 consumption is sum(load[i] * distance to go from i to i+1) + (penalty if bus stops at stop i)
    @objective(model, Max, lambda * sum(z[i] * on_data[i] for i in 1:n) - (1 - lambda) * (sum(load[i] * dist[i, i+1] for i in 1:n-1) + sum(z[i] for i in 1:n)))

    optimize!(model);

    return z, load, off, objective_value(model)
end

function bus_model_consecutive_constraint(on_data, off_data, load_data, x, y, n, dist, lambda)
    # function to optimize the bus route for a given lambda

    model = Model(Gurobi.Optimizer)
    set_attribute(model, "OutputFlag", 0)

    @variable(model, z[1:n], Bin)
    @variable(model, load[1:n] >= 0)
    @variable(model, off[1:n] >= 0)

    @constraint(model, [i in 1:n], load[i] <= 40)
    # load at bus stop j = load at bus stop j-1 + z_j * (number_go_on_at_stop_j - number_go_off_at_j-1)
    @constraint(model, load[1] == load_data[1])
    @constraint(model, [i in 1:n-1], z[i] + z[i+1] >= 1) # ADDED CONSTRAINT TO NOT ALLOW CONSECUTIVE STOPS TO DROP
    @constraint(model, load[n] <= 1)    # not sure here? equal to load_data or not?
    @constraint(model, [i in 2:n], load[i] == load[i-1] + z[i] * (on_data[i] - off[i-1]))
    # If a stop is removed, people go go off at the next available stop off_j = off_data_j + (1-z_{j-1})*off_j-1
    @constraint(model, off[1] == off_data[1])
    @constraint(model, [i in 2:n], off[i] == off_data[i] + (1-z[i-1]) * off[i-1])
    # bus has to stop at first and last stop
    @constraint(model, z[1] == 1)
    @constraint(model, z[n] == 1)

    # CO2 consumption is sum(load[i] * distance to go from i to i+1) + (penalty if bus stops at stop i)
    @objective(model, Max, lambda * sum(z[i] * on_data[i] for i in 1:n) - (1 - lambda) * (sum(load[i] * dist[i, i+1] for i in 1:n-1) + sum(z[i] for i in 1:n)))

    optimize!(model);

    return z, load, off, objective_value(model)
end

function model_stats(z, load, load_data, on_data, off, dist, n)
    # get absolute value and round to integer to account for cases where there's a floating point error
    z_opt = abs.(round.(value.(z)))

    println("\nNumber of stops: ", sum(z_opt))
    println(z_opt)

    load_opt = value.(load)
    println("Load at each stop: ", load_opt)
    off_opt = value.(off)
    println("Number of people going off at each stop: ", off_opt)

    current_co2 = sum(load_data[i] * dist[i, i+1] for i in 1:n-1) + n
    model_co2 = sum(load_opt[i] * dist[i, i+1] for i in 1:n-1) + sum(z_opt[i] for i in 1:n)
    current_profit = sum(on_data[i] for i in 1:n)
    model_profit = sum(z_opt[i] * on_data[i] for i in 1:n)

    println("\nCurrent CO2 emissions: ", sum(load_data[i] * dist[i, i+1] for i in 1:n-1) + n)
    println("With our model: ", sum(load_opt[i] * dist[i, i+1] for i in 1:n-1) + sum(z_opt[i] for i in 1:n))
    # % reduction (keep 1 decimal)

    println("Current profit: ", sum(on_data[i] for i in 1:n))
    println("With our model: ", sum(z_opt[i] * on_data[i] for i in 1:n))

    # % increase
    println("CO2 reduction: ", round((current_co2 - model_co2) / current_co2 * 100, digits=1), "%")
    println("Profit decrease: ", abs(round((model_profit - current_profit) / current_profit * 100, digits=1)), "%")

end

function plot_pareto(results_df, route_id, type)
    # plot co2 values and profit values
    plot(results_df[:, 4], results_df[:, 3], title="Pareto-Optimal Frontier for Route " * string(route_id), xlabel="Profit", ylabel="CO2 emissions", label="CO2 emissions vs Profit", legend=:topleft, seriestype=:scatter)
    savefig("key_routes_grouped_results_plots/pareto_route_" * string(route_id) * type * ".png") # save the most recent fig as filename_string (such as "output.png")
end

function plot_profit_co2_lambda(results_df, route_id, type)
    # plot profit, co2, and lambda at once
    plot(results_df[:, 1], results_df[:, 4], title = "Profit and CO2 vs Lambda for Route " * string(route_id), xlabel="Lambda", ylabel="Calue", label="Profit", legend=:outerright, seriestype=:scatter)
    plot!(results_df[:, 1], results_df[:, 3], xlabel="Lambda", ylabel="Value", label="CO2 emissions", legend=:bottomright, seriestype=:scatter)
    savefig("key_routes_grouped_results_plots/profit_co2_lambda_route_" * string(route_id) * type * ".png") # save the most recent fig as filename_string (such as "output.png")

end

function plot_stops_lambda(results_df, route_id, type)
    # plot stops and lambda
    plot(results_df[:, 1], results_df[:, 2], title = "Number of Stops vs Lambda for Route "  * string(route_id), xlabel="Lambda", ylabel="Number of stops", label="Number of stops vs Lambda", legend=:bottomright, seriestype=:scatter)
    savefig("key_routes_grouped_results_plots/stops_lambda_route_" * string(route_id) * type * ".png") # save the most recent fig as filename_string (such as "output.png")

end

function all_plots(route_id, type)
    # function to plot all plots

    # decide what to do based on what type
    if type == "general"
        suffix = ""
        results_df = CSV.read("key_routes_grouped_results/route_" * string(route_id) * "_opt_results.csv", DataFrame)
    else
        suffix = "_no_consecutive_drops"
        results_df = CSV.read("key_routes_grouped_results/route_" * string(route_id) * "_opt_results_no_consecutive_drops.csv", DataFrame)
    end

    # build all plots
    plot_pareto(results_df, route_id, suffix)
    plot_profit_co2_lambda(results_df, route_id, suffix)
    plot_stops_lambda(results_df, route_id, suffix)
end

all_plots (generic function with 2 methods)

### Run models, produce plots, store models and plots

In [236]:
# list of key routes
key_routes = [1, 15, 22, 23, 28, 32, 39, 57, 66, 71, 73, 77, 111, 116, 117]
done = [1, 22, 28, 32, 39, 66, 71, 73, 77, 111, 116, 117]
bad_routes = [15, 23, 57]

# run models for all lambdas and all routes
# get plots for all routes
for route_id in key_routes
    results_df, data = model_route(route_id, "general", false);
    all_plots(route_id, "general")
    #all_plots(route_id, "no_consecutive_drops")
end

optimizing route 1
on data: [7.815639875382104, 0.22256525862000115, 0.2948368739824316, 0.09937704244135431, 0.2980661674397016, 2.434997609212356, 2.9925134485189386, 1.7368405995700982, 1.1200957328705001, 2.250773007405078, 1.880222159587015, 1.4097579255873876, 5.2336585418808195, 1.1558360259590446, 0.9703722315453004, 0.532178062427265, 0.5073935902071749, 1.2359698027794241, 0.34675663567667975, 0.15436226833942382, 0.09275494289027451, 0.08685907255625334, 0.028750848937193818, 0.0]
off data: [0.0, 0.020638600453311028, 0.1512225811336891, 0.03974339629525387, 0.1732594315173591, 2.0562782957817674, 1.2110266669617917, 0.3300523649280569, 0.45562931951089847, 2.1971607111663625, 0.8987239113985294, 0.8841491921547516, 2.322157909842847, 0.24006590465838526, 1.6674100616194159, 1.5137187201898952, 1.6952882392787645, 4.9159489718913685, 0.7239669429384699, 0.7753730496838437, 1.06416530340327, 1.3913511947236972, 1.1144974305914592, 6.834475477132031]
load data: [7.967361158211

In [237]:
key_routes = [57, 66, 71, 73, 77, 111, 116, 117]
done = [1, 15, 28, 32, 39]
bad_routes = [22, 23]

# run models for all lambdas and all routes
# get plots for all routes
for route_id in key_routes
    results_df, data = model_route(route_id, "no_consecutive_drops", false);
    all_plots(route_id, "no_consecutive_drops")
end

optimizing route 57
on data: [0.9, 21.45, 0.3, 0.1, 0.1, 9.791324179946464, 1.6136387713967981, 1.0781002332118452, 1.584964667785419, 1.2008916005962527, 1.0531288754236499, 0.7185881370435611, 0.7658315118489637, 0.4266420248523359, 0.48333328839312084, 2.7944748158923463, 1.6135628172518877, 0.28228761666246716, 0.16756475255896872, 0.3511834335610761, 0.10142215699914149, 0.3777454558250331, 0.5350242380719048, 0.2465937835488095, 0.24655744014439668, 0.07781023768369202, 0.22485177413996713, 0.1327768329965178, 0.03667712059672073, 0.04832512748575273, 0.0041781135835533495, 0.014679123202507726, 0.007748111516709434, 0.021265587523764642, 0.06146984223395275, 0.007531605360210258, 0.0]
off data: [0.0, 1.75, 0.0, 0.0, 0.05, 0.0, 0.05393718313829501, 0.07326365138650745, 0.0950313542715318, 0.0913250662095148, 0.7265633862104741, 0.8497455057352348, 0.5694041682622655, 0.3488995683990698, 1.2002481496144244, 1.5871021200135687, 1.7822170017191954, 0.5765950117116846, 1.865968468352

### Other

In [235]:
# for one route
route_id = 1

# optimize and get plots
results_df, data = model_route(route_id, "both", false)
all_plots(route_id, "general")
all_plots(route_id, "no_consecutive_drops")

optimizing route 1
on data: [7.815639875382104, 0.22256525862000115, 0.2948368739824316, 0.09937704244135431, 0.2980661674397016, 2.434997609212356, 2.9925134485189386, 1.7368405995700982, 1.1200957328705001, 2.250773007405078, 1.880222159587015, 1.4097579255873876, 5.2336585418808195, 1.1558360259590446, 0.9703722315453004, 0.532178062427265, 0.5073935902071749, 1.2359698027794241, 0.34675663567667975, 0.15436226833942382, 0.09275494289027451, 0.08685907255625334, 0.028750848937193818, 0.0]
off data: [0.0, 0.020638600453311028, 0.1512225811336891, 0.03974339629525387, 0.1732594315173591, 2.0562782957817674, 1.2110266669617917, 0.3300523649280569, 0.45562931951089847, 2.1971607111663625, 0.8987239113985294, 0.8841491921547516, 2.322157909842847, 0.24006590465838526, 1.6674100616194159, 1.5137187201898952, 1.6952882392787645, 4.9159489718913685, 0.7239669429384699, 0.7753730496838437, 1.06416530340327, 1.3913511947236972, 1.1144974305914592, 6.834475477132031]
load data: [7.967361158211

"/Users/haydenratliff/Documents/School/MIT/Fall/15.093 Optimization Methods/bus_routing_project/key_routes_grouped_results_plots/stops_lambda_route_1_no_consecutive_drops.png"